# COVID-19 - analysis and prediction

## Checking the data
First, it is required to analyse our dataset so that it is sure to not have any problems. To do so, we will print the first entries of the dataset:

In [5]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
data = pd.read_csv('data/covid19.csv', parse_dates=['Date'])
data.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0,0


The first problem we can identify is that some entries don't have a value on the *Country/Region* column.

We can fix this by assigning the value 'NA' to those entries:

In [8]:
# filling missing values 
data[['Province/State']] = data[['Province/State']].fillna('')
data.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,,Afghanistan,33.0000,65.0000,2020-01-22,0,0,0
1,,Albania,41.1533,20.1683,2020-01-22,0,0,0
2,,Algeria,28.0339,1.6596,2020-01-22,0,0,0
3,,Andorra,42.5063,1.5218,2020-01-22,0,0,0
4,,Angola,-11.2027,17.8739,2020-01-22,0,0,0


We can now take a look at some summary statistics about the data set:

In [22]:
data.describe()

,Lat,Long,Confirmed,Deaths,Recovered
count,24890.000000,24890.000000,24890.000000,24890.000000,24890.000000
mean,21.433571,22.597991,2336.763198,141.207915,580.246605
std,24.740917,70.570914,22688.167259,1499.521732,5021.307422
min,-51.796300,-135.000000,-1.000000,-1.000000,0.000000
25%,7.000000,-19.020800,0.000000,0.000000,0.000000
50%,23.659750,20.921188,5.000000,0.000000,0.000000
75%,41.204400,81.000000,176.000000,2.000000,16.000000
max,71.706900,178.065000,938154.000000,53755.000000,109800.000000


We see that the count is the same for values in the columns *Lat*, *Long*, *Confirmed*, *Deaths* and *Recovered*.

There is, however, something strange with this dataset: The minimum value for *Confirmed* and *Deaths* is -1. Because negative values don't make sense for these two columns, we will use **mean imputation** to replace them with the average number for each column.

In [23]:
average_confirmed_count = data['Confirmed'].mean()
data.loc[(data['Confirmed'] < 0), 'Confirmed'] = average_confirmed_count

average_death_count = data['Deaths'].mean()
data.loc[(data['Deaths'] < 0), 'Deaths'] = average_death_count

data.describe()

,Lat,Long,Confirmed,Deaths,Recovered
count,24890.000000,24890.000000,24890.000000,24890.000000,24890.000000
mean,21.433571,22.597991,2338.359903,141.299330,580.246605
std,24.740917,70.570914,22688.084938,1499.517394,5021.307422
min,-51.796300,-135.000000,0.000000,0.000000,0.000000
25%,7.000000,-19.020800,0.000000,0.000000,0.000000
50%,23.659750,20.921188,6.000000,0.000000,0.000000
75%,41.204400,81.000000,178.000000,2.000000,16.000000
max,71.706900,178.065000,938154.000000,53755.000000,109800.000000


Now  those columns make more sense.

Also, to ensure data integrity, we will replace any missing values with 0s.

In [24]:
data[['Confirmed', 'Deaths', 'Recovered']] = data[['Confirmed', 'Deaths', 'Recovered']].fillna(0)

It would be a shame to lose all this tidied up data. Let's save it.

In [25]:
data.to_csv('data/covid19_clean.csv', index=False)

We'll now take a look at the scatterplot matrix now that we've tidied the data:

In [35]:
data_clean = pd.read_csv('data/covid19_clean.csv')
data_clean.fillna(data_clean.mean(), inplace=True)
# sb.pairplot(data.dropna())

## Classification

In [36]:
inputs = data_clean[['Lat', 'Long', 'Confirmed', 'Deaths', 'Recovered']].values

In [37]:
labels = data_clean[['Province/State', 'Country/Region', 'Date']].values